<a href="https://colab.research.google.com/github/tutumomo/autogen/blob/main/notebook_%E5%AF%A6%E4%BD%9C/autogen_function_call.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet --upgrade pyautogen

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [ ]:
import os

os.environ['OAI_CONFIG_LIST'] ='''[{"model": "gpt-4-1106-preview","api_key": "sk-OPENAI_API_KEY"},
{"model": "accounts/fireworks/models/fw-function-call-34b-v0","api_key": "FIREWORKS_API_KEY", "base_url":"https://api.fireworks.ai/inference/v1"},
{"model": "accounts/fireworks/models/mixtral-8x7b-instruct","api_key": "FIREWORKS_API_KEY", "base_url":"https://api.fireworks.ai/inference/v1"}]'''


import autogen

llm_config={
    "timeout": 600,
    "cache_seed": 57,  # change the seed for different trials
    "config_list": autogen.config_list_from_json(
        "OAI_CONFIG_LIST",
        filter_dict={"model": ["accounts/fireworks/models/fw-function-call-34b-v0"]},
        ),
    "temperature": 0.5,
}

## Making Function Calls


In [ ]:
chatbot = autogen.AssistantAgent(
    name="chatbot",
    system_message="""For currency exchange tasks,
    only use the functions you have been provided with.
    Reply TERMINATE when the task is done.
    Reply TERMINATE when user's content is empty.""",
    llm_config=llm_config,
)

# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().find("TERMINATE") >= 0,
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
)
from typing import Literal
CurrencySymbol = Literal["USD", "EUR"]


def exchange_rate(base_currency: CurrencySymbol, quote_currency: CurrencySymbol) -> float:
    if base_currency == quote_currency:
        return 1.0
    elif base_currency == "USD" and quote_currency == "EUR":
        return 1 / 1.1
    elif base_currency == "EUR" and quote_currency == "USD":
        return 1.1
    else:
        raise ValueError(f"Unknown currencies {base_currency}, {quote_currency}")


from pydantic import BaseModel, Field
from typing_extensions import Annotated

class Currency(BaseModel):
    currency: Annotated[CurrencySymbol, Field(..., description="Currency symbol")]
    amount: Annotated[float, Field(..., description="Amount of currency", ge=0)]

@user_proxy.register_for_execution()
@chatbot.register_for_llm(description="Currency exchange calculator.")
def currency_calculator(
    base: Annotated[Currency, "Base currency: amount and currency symbol"],
    quote_currency: Annotated[CurrencySymbol, "Quote currency symbol"] = "USD",
) -> Currency:
    quote_amount = exchange_rate(base.currency, quote_currency) * base.amount
    return Currency(amount=quote_amount, currency=quote_currency)

In [ ]:
chatbot.llm_config["tools"]

[{'type': 'function',
  'function': {'description': 'Currency exchange calculator.',
   'name': 'currency_calculator',
   'parameters': {'type': 'object',
    'properties': {'base': {'$ref': '#/definitions/Currency',
      'definitions': {'Currency': {'title': 'Currency',
        'type': 'object',
        'properties': {'currency': {'title': 'Currency',
          'description': 'Currency symbol',
          'enum': ['USD', 'EUR'],
          'type': 'string'},
         'amount': {'title': 'Amount',
          'description': 'Amount of currency',
          'minimum': 0,
          'type': 'number'}},
        'required': ['currency', 'amount']}},
      'description': 'Base currency: amount and currency symbol'},
     'quote_currency': {'enum': ['USD', 'EUR'],
      'type': 'string',
      'default': 'USD',
      'description': 'Quote currency symbol'}},
    'required': ['base']}}}]

In [ ]:
# 1. help function, move to the top of conversable_agent.py
def del_name(data):
  for item in data:
      if 'tool_call_id' in item:
          item.pop('name', None)
          break
# 2. call del_name(all_messages) in def generate_oai_reply
#    before response = client.create(

In [ ]:
# start the conversation
user_proxy.initiate_chat(
    chatbot,
    message="How much is 112.23 Euros in US Dollars?",
)

user_proxy (to chatbot):

How much is 112.23 Euros in US Dollars?

--------------------------------------------------------------------------------
chatbot (to user_proxy):

***** Suggested tool Call (call_zMg2lTOMEVWz0YVDP8FIKL2M): currency_calculator *****
Arguments: 
{"base": {"currency": "EUR", "amount": 112.23}, "quote_currency": "USD"}
************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION currency_calculator...
user_proxy (to chatbot):

user_proxy (to chatbot):

***** Response from calling tool "currency_calculator" *****
{"currency": "USD", "amount": 123.45300000000002}
************************************************************

--------------------------------------------------------------------------------
chatbot (to user_proxy):

112.23 Euros is approximately 123.45 US Dollars. 

------------------------------------------------------

In [ ]:
# start the conversation
user_proxy.initiate_chat(
    chatbot,
    message="How much is 112.23 Euros in US Dollars?",
)

user_proxy (to chatbot):

How much is 112.23 Euros in US Dollars?

--------------------------------------------------------------------------------
chatbot (to user_proxy):

***** Suggested tool Call (call_zMg2lTOMEVWz0YVDP8FIKL2M): currency_calculator *****
Arguments: 
{"base": {"currency": "EUR", "amount": 112.23}, "quote_currency": "USD"}
************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION currency_calculator...
user_proxy (to chatbot):

user_proxy (to chatbot):

***** Response from calling tool "currency_calculator" *****
{"currency": "USD", "amount": 123.45300000000002}
************************************************************

--------------------------------------------------------------------------------
chatbot (to user_proxy):

112.23 Euros is approximately 123.45 US Dollars. 

------------------------------------------------------